In [0]:
from IPython.display import clear_output

In [0]:
!pip3 install pycodestyle flake8 pycodestyle_magic
!pip3 install wldhx.yadisk-direct
clear_output()

In [0]:
%load_ext pycodestyle_magic

In [0]:
!wget https://github.com/ufal/udpipe/releases/download/v1.2.0/udpipe-1.2.0-bin.zip
clear_output()

In [0]:
!unzip udpipe-1.2.0-bin.zip
!rm udpipe-1.2.0-bin.zip
clear_output()

In [0]:
!wget https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-2998/russian-syntagrus-ud-2.4-190531.udpipe
clear_output()

In [0]:
!wget https://raw.githubusercontent.com/sjut/HSE-Compling/master/hw/testset2.txt
clear_output()

In [0]:
!curl -L $(yadisk-direct https://disk.yandex.ru/d/5WWwOr9ccemcZA) -o verb_coll.txt
clear_output()

In [0]:
!udpipe-1.2.0-bin/bin-linux64/udpipe --input horizontal --output conllu \
--tokenize --tag --parse \
russian-syntagrus-ud-2.4-190531.udpipe \
< testset2.txt > testset2.conllu
clear_output()

In [0]:
import re
from collections import Counter
from scipy.stats import spearmanr
from nltk.parse import DependencyGraph
from nltk.collocations import BigramAssocMeasures
from nltk.collocations import BigramCollocationFinder

# **Извлечение биграмм**

In [0]:
graphs = []

with open('testset2.conllu') as f:
    parsed_sents = f.read().split('\n\n')

    for sent in parsed_sents:
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        try:
            graphs.append(DependencyGraph('\n'.join(tree),
                                          top_relation_label='root'))
        # исключаем предложения, которые nltk не может распарсить из-за ошибки
        except AssertionError:
            continue

In [0]:
freq = Counter()

for g in graphs:
    freq.update([g.nodes[node]['lemma'] for node in g.nodes])

In [0]:
bigrams = []

for g in graphs:
    for node in g.nodes:
        # ищем пары вершина-зависимое VERB + NOUN в Acc
        # исключаем словосочетания с предлогами
        # и словосочетания со словом "раз"
        # т. к., на них находятся пары типа "делать (каждый) раз"
        if g.nodes[node]['ctag'] == 'NOUN' and re.match(
            r'.*Case=Acc.*', g.nodes[node]['feats']
            ) and g.nodes[node]['head'] and \
                  g.nodes[g.nodes[node]['head']]['ctag'] == 'VERB' and \
                  freq[g.nodes[g.nodes[node]['head']]['lemma']] >= 50 and \
                  not g.nodes[node]['deps']['case'] and \
                  not g.nodes[node]['word'] == 'раз':
            bigrams.append((g.nodes[g.nodes[node]['head']]['lemma'],
                             g.nodes[node]['word']))

In [14]:
len(bigrams)

2004

In [15]:
Counter(bigrams).most_common(10)

[(('подать', 'иск'), 75),
 (('принять', 'решение'), 54),
 (('просить', 'суд'), 46),
 (('удовлетворить', 'иск'), 46),
 (('вынести', 'решение'), 43),
 (('обжаловать', 'решение'), 40),
 (('вынести', 'приговор'), 32),
 (('удовлетворить', 'ходатайство'), 26),
 (('отменить', 'решение'), 23),
 (('подать', 'жалобу'), 21)]

In [0]:
finder = BigramCollocationFinder.from_documents(bigrams)

## **Метрики**

In [0]:
bigram_measures = BigramAssocMeasures()

In [18]:
scores_likelihood_ratio = finder.score_ngrams(bigram_measures.likelihood_ratio)
top100_likelihood_ratio = finder.nbest(bigram_measures.likelihood_ratio, 100)
top100_likelihood_ratio[:10]

[('просить', 'суд'),
 ('подать', 'иск'),
 ('принять', 'решение'),
 ('вынести', 'приговор'),
 ('обжаловать', 'решение'),
 ('удовлетворить', 'иск'),
 ('удовлетворить', 'ходатайство'),
 ('вынести', 'решение'),
 ('иметь', 'право'),
 ('отклонить', 'жалобу')]

In [19]:
scores_pmi = finder.score_ngrams(bigram_measures.pmi)
top100_pmi = finder.nbest(bigram_measures.pmi, 100)
top100_pmi[:10]

[('газета', 'предупреждение'),
 ('говорить', 'друга'),
 ('нет', 'пос'),
 ('обратиться', 'ответ'),
 ('отказаться', 'падение'),
 ('рассказать', 'замгендиректора'),
 ('связать', 'подзащитного'),
 ('и', 'о.'),
 ('пояснить', 'причину'),
 ('пояснить', 'суть')]

In [20]:
scores_dice = finder.score_ngrams(bigram_measures.dice)
top100_dice = finder.nbest(bigram_measures.dice, 100)
top100_dice[:10]

[('газета', 'предупреждение'),
 ('говорить', 'друга'),
 ('нет', 'пос'),
 ('обратиться', 'ответ'),
 ('отказаться', 'падение'),
 ('рассказать', 'замгендиректора'),
 ('связать', 'подзащитного'),
 ('просить', 'суд'),
 ('и', 'о.'),
 ('напомнить', 'основания')]

# **"Золотой стандарт" коллокаций**

In [21]:
top_100 = set(top100_likelihood_ratio) & set(top100_pmi) & set(top100_dice)
top_100

{('арестовывать', 'счета'),
 ('взыскать', 'задолженность'),
 ('взыскать', 'неустойку'),
 ('доказать', 'невиновность'),
 ('доказать', 'незаконность'),
 ('запретить', 'деятельность'),
 ('заявить', 'отвод'),
 ('назначить', 'наказание'),
 ('напомнить', 'основания'),
 ('объявить', 'голодовку'),
 ('объявить', 'перерыв'),
 ('пройти', 'прения'),
 ('решить', 'проблему'),
 ('решить', 'проблемы'),
 ('счесть', 'доводы'),
 ('требовать', 'Обвинение')}

In [0]:
with open('verb_coll.txt', 'r') as file:
    lines = [line.split() for line in file.read().splitlines()[:-1]]
    verb_coll = {(line[-2], line[-1]) for line in lines}

In [23]:
best_coll = top_100 & verb_coll
best_coll

{('объявить', 'перерыв'), ('решить', 'проблему')}

# **Улучшение "золотого стандарта" вручную**
Следующие биграммы из топ-100 не попали в словарь, но, тем не менее, являются коллокациями – т. е. являются устойчивыми словосочетаниями, семантически связаны, существительное биграммы представляет собой единственный внутренний аргумент глагола.

In [0]:
additional_coll = {
    ('арестовывать', 'счета'),
    ('взыскать', 'задолженность'),
    ('взыскать', 'неустойку'),
    ('доказать', 'невиновность'),
    ('доказать', 'незаконность'),
    ('запретить', 'деятельность'),
    ('назначить', 'наказание'),
    ('объявить', 'голодовку'),
}

In [25]:
best_coll = best_coll | additional_coll
best_coll

{('арестовывать', 'счета'),
 ('взыскать', 'задолженность'),
 ('взыскать', 'неустойку'),
 ('доказать', 'невиновность'),
 ('доказать', 'незаконность'),
 ('запретить', 'деятельность'),
 ('назначить', 'наказание'),
 ('объявить', 'голодовку'),
 ('объявить', 'перерыв'),
 ('решить', 'проблему')}

# **Ранговая корреляция результатов метрик для ЗС**

In [26]:
best_likelihood_ratio = [score for score in scores_likelihood_ratio
                         if score[0] in best_coll]
print(spearmanr(best_likelihood_ratio))

SpearmanrResult(correlation=0.6565379871744699, pvalue=0.039204386332556754)


In [27]:
best_pmi = [score for score in scores_pmi if score[0] in best_coll]
print(spearmanr(best_pmi))

SpearmanrResult(correlation=0.7744046403152623, pvalue=0.008543177424467151)


In [28]:
best_dice = [score for score in scores_dice if score[0] in best_coll]
print(spearmanr(best_dice))

SpearmanrResult(correlation=0.6890371996505876, pvalue=0.027533181700364857)


Самый высокий коэффициент корреляции Спирмена у результатов *PMI*, затем идёт *log-likelihood*, а хуже всех скоррелированы результаты *dice*.